In [827]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.DataFrame(pd.read_csv('March iHeart Everyone HR Pull.xlsx - Sheet1.csv'))
empl_roll = pd.DataFrame(pd.read_csv('CR Indexed Worker Roster - Sales 2021-01-04 19_16 CST.xlsx - Sheet1.csv'))
user_df = pd.DataFrame(pd.read_csv('ihm_users.csv'))
market_owners = pd.DataFrame(pd.read_csv('ihm_account_management.csv'))
workday = pd.DataFrame(pd.read_csv('usersChanges_2021_02_15_11_54.csv'))
revenue = pd.DataFrame(pd.read_csv('report1619030679059.csv'))
projects = pd.DataFrame(pd.read_csv('iheart_projects.csv')).loc[pd.DataFrame(pd.read_csv('iheart_projects.csv')).create_ts > '20200207']
newhr = pd.DataFrame(pd.read_csv('March iHeart Everyone HR Pull.xlsx - Sheet1.csv'))
accounts = pd.DataFrame(pd.read_csv('report1618865683080.csv'))['Account Name']
print('The shape of revenue is: ' + str(revenue.shape))

The shape of revenue is: (19275, 16)


In [828]:
def Differences(hrnames, accountnames):
    li_diff = [i for i in hrnames + accountnames if i not in hrnames or i not in accountnames]
    missingfromhr = [i for i in hrnames + accountnames if i not in hrnames]
    missingfromaccountnames = [i for i in hrnames + accountnames if i not in accountnames]
    if len(missingfromaccountnames) != 0:
        return 'missing from Salesforce account names:', missingfromaccountnames
    else:
        return print('full agreement')

Altering field names of tables to match Salesforce fields

In [829]:
empl_roll.rename(columns = {'Worker': 'hr_worker', 'Job Title': 'hr_job_title', 'Business Title': 'hr_business_title',
                                       'Hire Date': 'hr_hire_date', 'Phone - Primary Work': 'hr_work_phone',
                                       'Mobile Phones': 'hr_mobile_phone', "Worker's Manager": 'hr_worker_manager',
                                        "Worker's Manager Business Title": "hr_worker_manager_business_title",
                                       "Worker's Manager's Manager": "hr_worker_manager_manager",
                                       "Worker's Manager's Manager Business Title": "hr_worker_manager_manager_business_title",
                                       'Market': 'hr_market', 'Market President': 'hr_market_president', 'Region/Area': 'hr_region_area',
                                       'Area/Region/Community President': 'hr_area_region_community_president', 'Division': 'hr_division',
                                       'Division President': 'hr_division_president'}, inplace=True)

user_df.rename(columns = {'client': 'adwire_client', 'group_x': 'adwire_group', 'name_x': 'adwire_name_x',
                                       'email': 'adwire_email', 'projects': 'adwire_projects', 'spend': 'adwire_spend',
                                       'project_ts': 'adwire_first_posting', 'group_cap': 'adwire_cap', 'owner': 'account_owner',
                                        'group_upper': 'adwire_group_upper'}, inplace=True)

workday.rename(columns = {'first name_x': 'workday_first_name', 
                                       'action': 'workday_action', 'client code': 'workday_client_code', 'first name_y': 'workday_first_name2',
                                       'last name': 'workday_last_name', 'group_y': 'workday_group_y'}, inplace=True)

revenue.rename(columns = {'Opportunity: Opportunity ID': 'iHeart Opportunity ID',
                            'Opportunity: Stage': 'Stage',
                           'Opportunity: Opportunity Owner: Market Division': 'Revenue Market Division',
                           'Opportunity: Opportunity Owner: Market Area': 'Revenue Market Area',
                           'Opportunity: Opportunity Owner: Home Operating Market': 'Account Name',
                           'Opportunity: Opportunity Owner: Full Name': 'iHeart Oppty Owner',
                           'OppOwnerEmail': 'email',
                           'New Business': 'new_business',
                           'Opportunity: Account Name: Account Name': 'opp_account_name',
                           'Opportunity: Opportunity Name': 'Opportunity Name',
                           'Adwire Project ID': 'project_id',
                           'Form Date': 'form_date',
                           'Opportunity: Created Date': 'Created Date',
                           'Opportunity: Close Date': 'Close Date',
                           'Opportunity: Gross Expected Revenue': 'Expected iHeart Revenue',
                           'Opportunity: Gross Amount': 'iHeart Gross Revenue',
                           'Form Created on Closed Opp': 'Created on Closed Opp',
                           'Opportunity: Opportunity Owner: Market Type': 'Revenue Market Type'}, inplace=True)

projects.rename(columns = {'project_id': 'projects_id', 'adwire_post_date': 'projects_post_date', 'create_ts': 'projects_post_date', 'name_y': 'projects_name_y', 'group': 'projects_group',
                                        'submissions': 'projects_submissions', 'opportynity_id': 'projects_opportunity_id',
                                       'credits_used': 'projects_credits_used', 'industry': 'projects_industry', 'tone': 'projects_tone'}, inplace=True)

In [830]:
# Aligning market names to match Salesforce account names

#############
newhr['User - Division'] = ['iHeart Media - ' + x.replace(' - Market', '') for x in newhr['User - Division']]
newhr.loc[(newhr['User - Division'] == 'iHeart Media - Inside Sales & Bus Development'), 'User - Division'] = 'iHeart Media - Inside Sales'

print('newhr: ' + str(Differences(newhr['User - Division'].unique().tolist(), accounts.tolist())))

#############
revenue['form_date'] = pd.to_datetime(revenue['form_date'], errors='coerce').dt.strftime('%Y%m%d')

re_outer = re.compile(r'([^A-Z ])([A-Z])')
re_inner = re.compile(r'\b[A-Z]+(?=[A-Z][a-z])')
revenue['Account Name'] = ['iHeart Media - ' + re_inner.sub(r'\g<0> ', re_outer.sub(r'\1 \2', x)) for x in revenue['Account Name'].astype(str)]
revenue['Account Name'] = [x[:-3] + ', ' + x[-2:] for x in revenue['Account Name']]

revenue.loc[(revenue['Account Name'] == 'iHeart Media - St Petersburg, FL'),'Account Name'] = 'iHeart Media - Inside Sales'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Fort Myers, FL'), 'Account Name'] = 'iHeart Media - Fort Myers/Naples, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Cedar Rapids, IA'), 'Account Name'] = 'iHeart Media - Cedar Rapids/Iowa City, IA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Panama City, FL'), 'Account Name'] = 'iHeart Media - Panama City/Tallahassee, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Roanoke, VA'), 'Account Name'] = 'iHeart Media - Roanoke/Lynchburg, VA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Sarasota, FL'), 'Account Name'] = 'iHeart Media - Sarasota/Bradenton, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Ashland, OH'),'Account Name'] = 'iHeart Media - Ashland/Mansfield, OH'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - New York, NY'),'Account Name'] = 'iHeart Media - New York City, NY'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Modesto, CA'),'Account Name'] = 'iHeart Media - Modesto/Stockton, CA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Hattiesburg, MS'),'Account Name'] = 'iHeart Media - Laurel, MS'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Albany, GA'),'Account Name'] = 'iHeart Media - Columbus, GA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Mc Allen, TX'),'Account Name'] = 'iHeart Media - McAllen, TX'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - , an'),'Account Name'] = 'iHeart Media - No Data'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Ta, pa'),'Account Name'] = 'iHeart Media - Tampa, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Bay Area, CA'),'Account Name'] = 'iHeart Media - San Francisco, CA'

print('revenue: ' + str(Differences(revenue['Account Name'].unique().tolist(), accounts.tolist())))

#############
user_df.group = ['iHeart Media - ' + x for x in user_df.group]
user_df.group = user_df.group.str.strip()

user_df.loc[(user_df.group == 'iHeart Media - St. Petersburg, FL | Inside Sales'), 'group'] = 'iHeart Media - Inside Sales'
user_df.loc[(user_df.group == 'iHeart Media - Lexington / Somerset, KY'), 'group'] = 'iHeart Media - Lexington/Somerset, KY'
user_df.loc[(user_df.group == 'iHeart Media - Roanoke / Lynchburg, VA'), 'group'] = 'iHeart Media - Roanoke/Lynchburg, VA'
user_df.loc[(user_df.group == 'iHeart Media - Modesto / Stockton, CA'), 'group'] = 'iHeart Media - Modesto/Stockton, CA'

print('user_df: ' + str(Differences(user_df.group.unique().tolist(), accounts.tolist())))

#############
projects.projects_group = ['iHeart Media - ' + x for x in projects.projects_group]
projects.projects_group = projects.projects_group.str.strip()

projects.loc[(projects.projects_group == 'iHeart Media - St. Petersburg, FL | Inside Sales'), 'projects_group'] = 'iHeart Media - Inside Sales'
projects.loc[(projects.projects_group == 'iHeart Media - Lexington / Somerset, KY'), 'projects_group'] = 'iHeart Media - Lexington/Somerset, KY'
projects.loc[(projects.projects_group == 'iHeart Media - Roanoke / Lynchburg, VA'), 'projects_group'] = 'iHeart Media - Roanoke/Lynchburg, VA'
projects.loc[(projects.projects_group == 'iHeart Media - Modesto / Stockton, CA'), 'projects_group'] = 'iHeart Media - Modesto/Stockton, CA'

print('projects: ' + str(Differences(projects.projects_group.unique().tolist(), accounts.tolist())))

full agreement
newhr: None
revenue: ('missing from Salesforce account names:', ['iHeart Media - No Data'])
user_df: ('missing from Salesforce account names:', ['iHeart Media - Unsorted Users'])
projects: ('missing from Salesforce account names:', ['iHeart Media - Dave Sonefeld', 'iHeart Media - Libby Hogan'])


In [831]:
# Essentially a VLOOKUP; backfilling Account Name and Revenue Market Area fields with information from newhr because many of these values are consistently absent from
# the revenue report alone.

merge_newhr = newhr.rename(columns = {'User - User Email': 'email'})
merge_newhr['email'] = merge_newhr.email.str.lower()

revenue = revenue.merge(merge_newhr[['email', 'User - Division', 'User - Division Parent']], on='email', how='left')
revenue['Account Name'] = np.where(revenue['Account Name'] == 'iHeart Media - No Data', revenue['User - Division'], revenue['Account Name'])
revenue['Revenue Market Area'] = np.where(revenue['Revenue Market Area'].isnull(), revenue['User - Division Parent'], revenue['Revenue Market Area'])

revenue.drop('User - Division', axis=1, inplace=True)
revenue.drop('User - Division Parent', axis=1, inplace=True)

Mutating 3 columns:
- Writelabel Owner - Opportunity Owner (revenue)
- Writelabel Owner - Project Owner (projects)
- Writelabel Owner - User (user_df)
- Writelabel Owner (wl_owner) - Employee (newhr)

In [832]:
arliss = market_owners[market_owners['Owner'] == 'Arliss Coates']
arliss = ['iheart media - ' + x.lower() for x in arliss.Name.tolist()]
ashley = market_owners[market_owners['Owner'] == 'Ashley McCarthy']
ashley = ['iheart media - ' + x.lower() for x in ashley['Name'].tolist()]
jenny = market_owners[market_owners['Owner'] == 'Jenny Donheiser']
jenny = ['iheart media - ' + x.lower() for x in jenny['Name'].tolist()]
mariana = market_owners[market_owners['Owner'] == 'Mariana E Trevino Stafford']
mariana = ['iheart media - ' + x.lower() for x in mariana['Name'].tolist()]

newhr_conditions = [(newhr['User - Division'].str.lower().isin(arliss)),
              (newhr['User - Division'].str.lower().isin(ashley)),
              (newhr['User - Division'].str.lower().isin(mariana)),
              (newhr['User - Division'].str.lower().isin(jenny))]

revenue_conditions = [(revenue['Account Name'].str.lower().isin(arliss)),
              (revenue['Account Name'].str.lower().isin(ashley)),
              (revenue['Account Name'].str.lower().isin(mariana)),
              (revenue['Account Name'].str.lower().isin(jenny))]

projects_conditions = [(projects.projects_group.str.lower().isin(arliss)),
              (projects.projects_group.str.lower().isin(ashley)),
              (projects.projects_group.str.lower().isin(mariana)),
              (projects.projects_group.str.lower().isin(jenny))]

user_df_conditions = [(user_df.group.str.lower().isin(arliss)),
              (user_df.group.str.lower().isin(ashley)),
              (user_df.group.str.lower().isin(mariana)),
              (user_df.group.str.lower().isin(jenny))]

values = ['Arliss Coates', 'Ashley McCarthy', 'Mariana Trevino', 'Jenny Donheiser']

newhr['wl_owner'] = np.select(newhr_conditions, values)
revenue['wl_owner'] = np.select(revenue_conditions, values)
projects['wl_owner'] = np.select(projects_conditions, values)
user_df['wl_owner'] = np.select(user_df_conditions, values)

In [839]:
# revenue.loc[(revenue['Form Date'] > '20210402') & (revenue['Form Date'] != 'NaT')]
# projects[projects.apply(lambda row: row.astype(str).str.contains('converse', case=False).any(), axis=1)]
newhr[newhr.apply(lambda row: row.astype(str).str.contains('converse', case=False).any(), axis=1)]
# revenue.shape

,First Name,Last Name,Name,User - Division Parent,User - Division,User - Position,User - User Email,User - Manager - User Full Name,User - Manager - User Email,wl_owner
1624,Lisa,Converse,Lisa Converse,Tampa - Area,"iHeart Media - Tampa, FL",Outside Account Executive - 100% Commission,LisaConverse@iheartmedia.com,"Robertson, Russell",RussellRobertson@iheartmedia.com,Mariana Trevino


In [834]:
newhr.head()

,First Name,Last Name,Name,User - Division Parent,User - Division,User - Position,User - User Email,User - Manager - User Full Name,User - Manager - User Email,wl_owner
0,Jeff,Rovinsky,Jeff Rovinsky,Cleveland - Area,"iHeart Media - Akron, OH",Metro President,JeffRovinsky@iheartmedia.com,"Hotchkiss, Keith",KeithHotchkiss@iheartmedia.com,Ashley McCarthy
1,Michelle,Allen,Michelle Allen,Cleveland - Area,"iHeart Media - Akron, OH",Outside Account Executive - 100% Commission,MichelleAllen@iheartmedia.com,"Rovinsky, Jeff",JeffRovinsky@iheartmedia.com,Ashley McCarthy
2,Jennifer,Grabiec,Jennifer Grabiec,Cleveland - Area,"iHeart Media - Akron, OH",Outside Account Executive - 100% Commission,JenniferGrabiec@iheartmedia.com,"Rovinsky, Jeff",JeffRovinsky@iheartmedia.com,Ashley McCarthy
3,Nichole,Gutshall,Nichole Gutshall,Cleveland - Area,"iHeart Media - Akron, OH",Outside Account Executive - 100% Commission,NicholeGutshall@iheartmedia.com,"Razzante, Matt",MattRazzante@iheartmedia.com,Ashley McCarthy
4,Andrea,Katusin,Andrea Katusin,Cleveland - Area,"iHeart Media - Akron, OH",Outside Account Executive - 100% Commission,AndreaKatusin@iheartmedia.com,"Razzante, Matt",MattRazzante@iheartmedia.com,Ashley McCarthy


In [835]:
revenue.head()

,iHeart Opportunity ID,Stage,Revenue Market Division,Revenue Market Area,Account Name,iHeart Oppty Owner,email,Opportunity Name,project_id,form_date,Created Date,Close Date,Expected iHeart Revenue,iHeart Gross Revenue,Created on Closed Opp,Revenue Market Type,wl_owner
0,0060y00001Fo10M,Closed - Won,NaN,Inside Sales & Business Development,iHeart Media - Inside Sales,James Boyer,jamesboyer@iheartmedia.com,Nymbl Scie-Schroeder-Inside Sales-9399588,B9ZgHBDwAAA,20210302,3/1/2021,3/3/2021,3402.0,3402.0,0,NaN,Ashley McCarthy
1,0060y00001FbyEZ,Closed - Lost,NaN,Inside Sales & Business Development,iHeart Media - Inside Sales,Jordan Lemire,jordanlemire@iheartmedia.com,All Is Wel-Teng-Inside Sales-9041721,BsFThGFQAAA,20201123,11/23/2020,2/10/2021,0.0,19380.0,0,NaN,Ashley McCarthy
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ac5iKAXAAAA,NaT,NaN,NaN,NaN,NaN,0,NaN,0
3,0060y00001ECIMf,Closed - Won,NaN,NaN,NaN,Jennifer Minigutti Mayfield,jennifermayfield@iheartmedia.com,AMERICAN F-Houston Test Market-MMS-Copy-8405929,A_PQMBrQAAA,20200710,6/24/2020,6/24/2020,65195.0,65195.0,1,NaN,0
4,0060y00001BJfCT,Closed - Won,NaN,Inside Sales & Business Development,iHeart Media - Inside Sales,Scott Schoenberg,scottschoenberg@iheartmedia.com,MMP-31480_VitaCoco_Tampa_Houston-Inside Sales-...,AVCPoABgAAA,20200304,12/6/2019,1/24/2020,147840.0,147840.0,1,NaN,Ashley McCarthy


In [836]:
projects.head()

,projects_id,projects_post_date,email,title,name,name.1,projects_group,projects_submissions,projects_opportunity_id,projects_credits_used,projects_industry,projects_tone,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,wl_owner
17959,B4c1_QDgAAA,2021-01-01 14:48:14,kariporter@iheartmedia.com,BREMER BANK INC,AdWire Draft Scripts | 15 AND 30 Seconds,Kari Porter,"iHeart Media - Minot, ND",6,0060y00001FeGnZAAV,2.5,Financial Services,Serious,NaN,NaN,NaN,NaN,NaN,Arliss Coates
17960,B4dAhDGwAAA,2021-01-01 15:33:09,kariporter@iheartmedia.com,Dakota Territory Air Museum,AdWire Draft Scripts | 15 AND 30 Seconds,Kari Porter,"iHeart Media - Minot, ND",5,0060y00001FeGm2AAF,2.5,Not-For-Profit/Charitable/Religious,Serious,NaN,NaN,NaN,NaN,NaN,Arliss Coates
17961,B4dBsQDgAAA,2021-01-01 15:38:10,kariporter@iheartmedia.com,Premier Limousine | weddings,AdWire Draft Scripts | 15 AND 30 Seconds,Kari Porter,"iHeart Media - Minot, ND",7,0060y00001FdjAqAAJ,2.5,Transportation Svcs,No preference. I'm open to a range of good ideas.,NaN,NaN,NaN,NaN,NaN,Arliss Coates
17962,B4dGiRBAAAA,2021-01-01 15:58:49,kariporter@iheartmedia.com,West Oaks Animal Hospital Inc | Awareness,AdWire Draft Scripts | 15 Seconds,Kari Porter,"iHeart Media - Minot, ND",6,0060y00001FeGnPAAV,1,Professional Services,No preference. I'm open to a range of good ideas.,NaN,NaN,NaN,NaN,NaN,Arliss Coates
17963,B4dItTGwAAA,2021-01-01 16:08:05,kariporter@iheartmedia.com,Preferred Partners Real Estate | 2021 Start Up,AdWire Draft Scripts | 15 Seconds,Kari Porter,"iHeart Media - Minot, ND",6,0060y00001FeGltAAF,1,Housing/Real Estate,No preference. I'm open to a range of good ideas.,NaN,NaN,NaN,NaN,NaN,Arliss Coates


In [837]:
user_df.head()

,adwire_client,group,name,adwire_email,adwire_projects,adwire_spend,adwire_first_posting,adwire_cap,wl_owner
0,iHeart Media,"iHeart Media - Tampa, FL",Kelly Thorson,kellythorson@iheartmedia.com,41,43.5,2018-10-23 14:55:53,35,Mariana Trevino
1,iHeart Media,"iHeart Media - Tampa, FL",Lisa Converse,lisaconverse@iheartmedia.com,30,32.0,2018-12-11 15:10:31,35,Mariana Trevino
2,iHeart Media,"iHeart Media - Tampa, FL",Mary Weber,maryweber@iheartmedia.com,10,15.5,2018-10-29 18:32:36,35,Mariana Trevino
3,iHeart Media,"iHeart Media - Tampa, FL",Michelle Land,michelleland@iheartmedia.com,21,24.0,2018-10-24 15:54:31,35,Mariana Trevino
4,iHeart Media,"iHeart Media - Tampa, FL",Shannon Gibson,shannongibson@iheartmedia.com,37,39.0,2018-10-26 12:58:28,35,Mariana Trevino
